In [1]:
import pandas as pd
import time


from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer
detector_classes = [DetectorDetectGPT]
explainer_classes = [  SHAP_Explainer]
results = []


In [2]:
test = pd.read_pickle("./dataset_test.pkl")
train = pd.read_pickle("./dataset_train.pkl")

In [3]:
len(test)+len(train)

1016

In [4]:
documents = test["answer"]
gold_labels = test["author"] == "human_answers"

In [5]:
from tqdm import tqdm

In [6]:
# for detector_class in detector_classes:
#     detector = detector_class()
#     start = time.time()
#     y = detector.predict_label(documents)
#     end = time.time()

#     acc = sum(y == gold_labels)/ len(documents)
#     results.append((detector.__class__.__name__, acc, end - start))
#     print(results[-1])


In [7]:
# detector = DetectorDetectGPT()

In [8]:
# detector.get_pad_token_id_masker()

In [9]:
import numpy as np

In [10]:
# y = detector.predict_proba(documents)
# yy = np.array(y).argmax(axis=1)
# acc = sum(yy == gold_labels)/ len(documents)
# acc

In [11]:
from IPython.core.display import display, HTML

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display


In [12]:
# for document, gt in zip(documents,gold_labels):
#     explainer = SHAP_Explainer(detector)
#     print(gt)
#     display(HTML(explainer.get_vanilla_visualization_HTML(document)))

In [13]:
# for document in tqdm(documents):
#     detector.predict_proba([document])

In [14]:
import shap

In [15]:
# for detector_class in detector_classes:
#     detector = detector_class()
#     for explainer_class in explainer_classes:
#         explainer = explainer_class(detector)
#         print(explainer.__class__.__name__, detector.__class__.__name__)
#         for document in tqdm(documents):
#             shap.plots.text(explainer.get_explanation(document))

In [16]:
detector = DetectorDetectGPT()


Using cache dir ./.cache
Loading BASE model EleutherAI/pythia-70m...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MOVING BASE MODEL TO GPU...DONE (2.21s)
DONE (0.06s)


In [17]:
# detector = DetectorRadford()
explainer = SHAP_Explainer(detector)

explainer.get_fi_scores_batch(documents)

  0%|          | 0/305 [00:06<?, ?it/s] | 0/351 [00:00<?, ?it/s]


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
explainer = LIME_Explainer(detector)

explainer.get_fi_scores_batch(documents)